## **RADI608: Data Mining and Machine Learning**

### Assignment: Model Deployment
**Romen Samuel Rodis Wabina** <br>
Student, PhD Data Science in Healthcare and Clinical Informatics <br>
Clinical Epidemiology and Biostatistics, Faculty of Medicine (Ramathibodi Hospital) <br>
Mahidol University

Note: In case of Python Markdown errors, you may access the assignment through this GitHub [Link](https://github.com/rrwabina/RADI608/tree/main/Submitted)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn import datasets

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import label_binarize
from sklearn.metrics import average_precision_score

import dataset as data

In [2]:
import pickle
  
def storeData():
    # initializing data to be stored in db
    Omkar = {'key' : 'Omkar', 'name' : 'Omkar Pathak',
    'age' : 21, 'pay' : 40000}
    Jagdish = {'key' : 'Jagdish', 'name' : 'Jagdish Pathak',
    'age' : 50, 'pay' : 50000}
  
    # database
    db = {}
    db['Omkar'] = Omkar
    db['Jagdish'] = Jagdish
      
    # Its important to use binary mode
    dbfile = open('examplePickle', 'ab')
      
    # source, destination
    pickle.dump(db, dbfile)                     
    dbfile.close()
  
def loadData():
    dbfile = open('examplePickle', 'rb')     
    db = pickle.load(dbfile)
    for keys in db:
        print(keys, '=>', db[keys])
    dbfile.close()
  
if __name__ == '__main__':
    storeData()
    loadData()

Omkar => {'key': 'Omkar', 'name': 'Omkar Pathak', 'age': 21, 'pay': 40000}
Jagdish => {'key': 'Jagdish', 'name': 'Jagdish Pathak', 'age': 50, 'pay': 50000}


In [3]:
import pickle 

db = {}
patient1 = {'FirstName': 'Leonel',
            'LastName' : 'Messi',
            'Height'   :  168}
patient2 = {'FirstName': 'Christiano',
            'LastName' : 'Ronaldo',
            'Height'   :  186}

db['001'] = patient1
db['002'] = patient2

dbfile = open('sampledb.pkl', 'wb')
pickle.dump(db, dbfile)
dbfile.close()

In [4]:
import warnings 
warnings.filterwarnings('ignore')

X, y = data.get_data()

def scale_data(X, y, test_size = 0.3):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size)
    return X_train, X_test, y_train, y_test

def create_intercepts(X_train, X_test):
    intercept = np.ones((X_train.shape[0], 1))
    X_train = np.concatenate((intercept, X_train), axis=1)  
    intercept = np.ones((X_test.shape[0], 1))
    X_test = np.concatenate((intercept, X_test), axis=1)
    return X_train, X_test 
 
def logistic(X_train, X_test, y_train):
    model = LogisticRegression(multi_class = 'ovr')  
    model.fit(X_train, y_train)
    yhat = model.predict(X_test)
    return yhat

def display_results(yhat, y_test):
    y_test_binarized = label_binarize(y_test, classes = [0, 1, 2])
    yhat_binarized   = label_binarize(yhat,   classes = [0, 1, 2])

    n_classes = len(np.unique(y_test))
    for i in range(n_classes):
        class_score = average_precision_score(y_test_binarized[:, i], yhat_binarized[:, i])
        print(f"Class {i} score: ", np.round(class_score, 3))
    print("Report: ", classification_report(y_test, yhat))

In [5]:
import pickle
def storeData():  
    column_names = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"]
    df = pd.read_csv("data/processed.cleveland.data", header = None, names = column_names)

    for column in column_names:
        df = df[(df[column] != "?")].reset_index(drop=True)

    obj_df = df.select_dtypes(include=['object']).copy()

    for column in column_names:
        df[column] = df[column].astype(float)

    df['label'] = df['num'].apply(lambda x: 0 if x == 0 else 1)
    df.drop('num', axis=1, inplace = True)

    # database
    columns = ['thal', 'cp', 'slope', 'exang', 'label']
    db = {}
    for column in columns:
        db[column] = df[columns]
    sample = open('disease.pkl', 'ab')
    pickle.dump(db, sample)                     
    sample.close()


def loadData():
    dbfile = open('disease.pkl', 'rb')     
    db = pickle.load(dbfile)
    for keys in db:
        print(keys, '=>', db[keys])
    dbfile.close()
storeData()

In [6]:
import logistic as log 

X, y = data.get_data()

X_train, X_test, y_train, y_test = log.scale_data(X, y)
X_train, X_test = log.create_intercepts(X_train, X_test)
yhat = log.logistic(X_train, X_test, y_train)

display_results(yhat, y_test)

Class 0 score:  0.767
Class 1 score:  0.551
Report:                precision    recall  f1-score   support

           0       0.83      0.70      0.76        56
           1       0.60      0.76      0.68        34

    accuracy                           0.72        90
   macro avg       0.72      0.73      0.72        90
weighted avg       0.74      0.72      0.73        90



In [7]:
# @app.route('/')
# def hello():
#     return     ''' 
#     <body> 
#             <h2> Hello Romen! </h2>
#     </body>
#     '''

# # @app.route('/greet/<name>')
# # def greet(name):
# #     return 'Hello {}!'.format(name)

In [8]:
import flask
app = flask.Flask(__name__)

heart = pickle.load(open('disease', 'rb'))

@app.route('/')
def home():
    return ''' 
            <body>
                <h1> Heart Disease Prediction </h1>
            </body>
           '''

@app.route('/predict', methods = ['GET'])
def predict():
    thal    = flask.request.args['thal']
    cp      = flask.request.args['cp']
    slope   = flask.request.args['slope']
    exang   = flask.request.args['exang']
    ca      = flask.request.args['ca']

    fmap = {'normal' : [1, 0, 0], 
            'fixed defect': [0, 1, 0], 
            'reversable defect': [0, 0, 1], 
            'typical angina': [1, 0, 0, 0],
            'atypical angina': [0, 1, 0, 0], 
            'non anginal pain': [0, 0, 1, 0],
            'asymptomatic': [0, 0, 0, 1],
            'upsloping': [1, 0, 0],
            'flat': [0, 1, 0],
            'downsloping': [0, 0, 1]}
    
    X_df = np.array(fmap[thal] + fmap[cp] + fmap[slope] + [int(exang)] +[int(ca)]).reshape(1, -1)
    yhat = heart.predict(X_df)
    if yhat[0] == 1:
        outcome = 'Heart Disease'
    else:
        outcome = 'Healthy!'
    probability = heart.predict_proba(X_df)
    return 'This patient is diagnosed with' + outcome + 'with probability' + np.round(probability[0][1], 2)

if __name__ == '__main__':
    HOST = '127.0.0.1'
    PORT = '4000'
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [13]:
import numpy as np
import flask
import pickle

app = flask.Flask(__name__)

heart = pickle.load(open("logregheart.pkl", "rb"))

@app.route("/")
def home():
    return """
           <body>
                    <h1> Heart Disease Prediction <h1>
           </body>
           """

@app.route("/predict", methods=["GET"])
def predict():
    thal    = flask.request.args['thal']
    cp      = flask.request.args['cp']
    slope   = flask.request.args['slope']
    exang   = flask.request.args['exang']
    ca      = flask.request.args['ca']

    
    fmap = {'normal' : [1, 0, 0], 
            'fixed defect': [0, 1, 0], 
            'reversable defect': [0, 0, 1], 
            'typical angina': [1, 0, 0, 0],
            'atypical angina': [0, 1, 0, 0], 
            'non anginal pain': [0, 0, 1, 0],
            'asymptomatic': [0, 0, 0, 1],
            'upsloping': [1, 0, 0],
            'flat': [0, 1, 0],
            'downsloping': [0, 0, 1]}
    
    X_new = np.array(fmap[thal] + fmap[cp] + fmap[slope] + [int(exang)] + [int(ca)]).reshape(1, -1)
    yhat = heart.predict(X_new)
    if yhat[0] == 1:
        outcome = "heart disease"
    else:
        outcome = "normal"
    prob = heart.predict_proba(X_new)
    return "This patient is diagnosed as " + outcome + " with probability " + str(round(prob[0][1], 2))


@app.route("/page")
def page():
   with open("page.html", 'r') as viz_file:
       return viz_file.read()

@app.route("/result", methods=["GET", "POST"])
def result():
    if flask.request.method == "POST":
        inputs = flask.request.form
        thal = inputs["thal"]
        cp = inputs["cp"]
        slope = inputs["slope"]
        exang = inputs["exang"]
        ca = inputs["ca"]
        
    fmap = {"normal": [1, 0, 0],
            "fixed defect": [0, 1, 0],
            "reversable defect": [0, 0, 1],
            "typical angina": [1, 0, 0, 0],
            "atypical angina": [0, 1, 0, 0],
            "non anginal pain": [0, 0, 1, 0],
            "asymptomatic": [0, 0, 0, 1],
            "upsloping": [1, 0, 0],
            "flat": [0, 1, 0],
            "downsloping": [0, 0, 1]}

    X_new = np.array(fmap[thal] + fmap[cp] + fmap[slope] + [int(exang)] + [int(ca)]).reshape(1, -1)
    yhat = heart.predict(X_new)

    if yhat[0] == 1:
        outcome = 'heart disease'
    else:
        outcome = 'healthy'

    prob = heart.predict_proba(X_new)

    results = """
              <body>
              <h3> Heart Disease Diagnosis <h3>
              <p><h4> Patient profile </h4></p>
              <table>
              <tr>
                  <td>Thalassemia: </td>
                  <td>""" + thal + """</td>
              </tr>
              <tr>
                  <td>Chest Pain: </td>
                  <td>""" + cp + """</td>
              </tr>
              <tr>
                  <td>Slope: </td>
                  <td>""" + slope + """</td>
              </tr>
              <tr>
                  <td>Exercise induced angina: </td>
                  <td>""" + exang + """</td>
              </tr>
              <tr>
                  <td>Number of major vessels</td>
                  <td>""" + ca + """</td>
              </tr>
              </table>
              <p> This patient is diagnose as """ + outcome + """ with probability """ + str(round(prob[0][1], 2)) + """.
              </body>"""

    return results
if __name__ == '__main__':
    HOST = "127.0.0.1"
    PORT = "4000"
    app.run(HOST, PORT)


UnpicklingError: invalid load key, '\xef'.

In [3]:
import flask
import pickle

app = flask.Flask(__name__)


@app.route('/')
def hello():
    return     ''' 
    <body> 
            <h2> Hello Romen! Hello guys  </h2>
    </body>
    '''


if __name__ == '__main__':
    HOST = "127.0.0.1"
    PORT = "4000"
    app.run(HOST, PORT)
# @app.route('/greet/<name>')
# def greet(name):
#     return 'Hello {}!'.format(name)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Dec/2022 10:17:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2022 10:17:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2022 10:17:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2022 10:18:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2022 10:18:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2022 10:18:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2022 10:18:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2022 10:18:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2022 10:18:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2022 10:18:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2022 10:18:24] "GET / HTTP/1.1" 200 -
